In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy.io import arff
from io import StringIO
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
dataset=pd.read_csv('wdbc.data',header=None)
dataset=dataset.iloc[:,1:]
dataset.iloc[:,0]= dataset.iloc[:,0].astype('category')
dataset.iloc[:,0]=dataset.iloc[:,0].cat.codes

# temp1=dataset.iloc[:,:-1]
# temp2=dataset.iloc[:,-1]
# # # temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
# dataset=pd.concat([temp2,temp1],axis=1)
original_data=dataset

original_data,testset= train_test_split(dataset, test_size=0.2)

In [5]:
original_data.head()

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
517,1,19.89,20.26,130.50,1214.0,0.10370,0.13100,0.14110,0.09431,0.1802,...,23.73,25.23,160.50,1646.0,0.1417,0.33090,0.4185,0.16130,0.2549,0.09136
400,1,17.91,21.02,124.40,994.0,0.12300,0.25760,0.31890,0.11980,0.2113,...,20.80,27.78,149.60,1304.0,0.1873,0.59170,0.9034,0.19640,0.3245,0.11980
69,0,12.78,16.49,81.37,502.5,0.09831,0.05234,0.03653,0.02864,0.1590,...,13.46,19.76,85.67,554.9,0.1296,0.07061,0.1039,0.05882,0.2383,0.06410
323,1,20.34,21.51,135.90,1264.0,0.11700,0.18750,0.25650,0.15040,0.2569,...,25.30,31.86,171.10,1938.0,0.1592,0.44920,0.5344,0.26850,0.5558,0.10240
502,0,12.54,16.32,81.25,476.3,0.11580,0.10850,0.05928,0.03279,0.1943,...,13.57,21.40,86.67,552.0,0.1580,0.17510,0.1889,0.08411,0.3155,0.07538


In [2]:
def orthogonal_sum(m_matrix): # input is a k*q matrix, k= # of nearest neighbors, q= # of classes
                              # e.g m_matrix=[[0.1,0.2,0.7],[0.50,0.5,0]]  
    (k,q)=m_matrix.shape
    M=m_matrix[0]
    for i in range(k-1):
        outer=np.outer(M,m_matrix[i+1])
        to_be_subtracted=np.sum(np.diag(outer))
        k=(np.sum(outer)-to_be_subtracted)/2.0
        norm=1-k
        M=M*m_matrix[i+1]/norm
    return M

In [3]:
def PCR5_sum(m_matrix):
    (k,q)=m_matrix.shape
    M=m_matrix[0]
    for i in range(k-1):
        M=M*m_matrix[i+1]+M**2*m_matrix[i+1]/(M+m_matrix[i+1])+M*m_matrix[i+1]**2/(M+m_matrix[i+1])
    return M

In [4]:
def PCR6_sum(m_matrix):
    (k,q)=m_matrix.shape
    M=m_matrix[0]
    for i in range(k-1):
        base=M*m_matrix[i+1]
        outer1=np.outer(M**2,m_matrix[i+1])
        diag1=np.diag(outer1)
        #print diag1
        sum1=np.sum(outer1,axis=1)-diag1
        sum2=[]

        for j in range(q):
            #print m_matrix[i+1,np.delete(np.arange(q),j)]
            temp=M[j]+m_matrix[i+1,np.delete(np.arange(q),j)]
            sum2.append(temp)
        sum2=np.sum(sum2,axis=1)

        outer2=np.outer(m_matrix[i+1]**2,M)
        diag2=np.diag(outer2)
        sum3=np.sum(outer2,axis=1)-diag2
        sum4=[]
        for j in range(q):
            #print m_matrix[i+1,np.delete(np.arange(q),j)]
            temp2=m_matrix[i+1,j]+M[np.delete(np.arange(q),j)]
            sum4.append(temp2)
        sum4=np.sum(sum4,axis=1)
        M=base+sum1/sum2+sum3/sum4

        
    return M

In [88]:
m_matrix=np.array([[0.1,0.2,0.7],[0.50,0.5,0]])
M=m_matrix[0]
(k,q)=m_matrix.shape
for i in range(k-1):
    base=M*m_matrix[i+1]
    outer1=np.outer(M**2,m_matrix[i+1])
    diag1=np.diag(outer1)
    #print diag1
    sum1=np.sum(outer1,axis=1)-diag1
    sum2=[]
        
    for j in range(q):
        #print m_matrix[i+1,np.delete(np.arange(q),j)]
        temp=M[j]+m_matrix[i+1,np.delete(np.arange(q),j)]
        sum2.append(temp)
    sum2=np.sum(sum2,axis=1)
    
    outer2=np.outer(m_matrix[i+1]**2,M)
    diag2=np.diag(outer2)
    sum3=np.sum(outer2,axis=1)-diag2
    sum4=[]
    for j in range(q):
        #print m_matrix[i+1,np.delete(np.arange(q),j)]
        temp2=m_matrix[i+1,j]+M[np.delete(np.arange(q),j)]
        sum4.append(temp2)
    sum4=np.sum(sum4,axis=1)
    M=base+sum1/sum2+sum3/sum4
    

In [5]:
def avg_sum(m_matrix):
    M=np.mean(m_matrix,axis=0)
    return M

In [6]:
def yager_sum(m_matrix):
    M=np.product(m_matrix,axis=0)
    return M

In [7]:

def combine_mass(m1,m2):
    M=1-(1-m1)*(1-m2)
    return (M)



In [8]:
# Calculating combined BPA m1
def find_mi(target,mass,i):
        
    Mass=0
    if target.iloc[0]==i:
        Mass=mass[0]
        if target.iloc[1]==i:
            Mass=combine_mass(Mass,mass[1])
        if target.iloc[2]==i:
            Mass=combine_mass(Mass,mass[2])
    return (Mass)


In [9]:
#### Back Up
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
import math
def DS_KNN(training,test,ave_dist,metric,rule): # training data should be dataframe and also include target
    label=[]
    X_test2=test.iloc[:,1:]
    processed_test=[]
    for i in range(len(X_test2)):
        processed_test.append(np.array(X_test2.iloc[i]))
    processed_test=np.array(processed_test)
    for pt in processed_test:
        dist=distance.cdist([pt], training.iloc[:,1:], metric)
        dist_order=np.argsort(dist)
        best_3_index=dist_order[0,:3]
        nn_3=training.iloc[best_3_index,1:]
        
        target=training.iloc[best_3_index,0]
        target=np.array(target)
        
        target=target.astype(int)
        gamma=ave_dist[target]# target is 0 and 1 so it is changed accordingly        
        mass_=0.95*np.exp(-1.0/gamma*dist[0,best_3_index]**2)
        k=3
        q=len(original_data.iloc[:,0].unique())
        mass_matrix=np.zeros((k,q))
        for i in range(len(mass_)):
            mass_matrix[i,[target[i]]]=mass_[i]
        
        if rule=='orthogonal':
            final_mass=orthogonal_sum(mass_matrix)
        elif rule=='yager':
            final_mass=yager_sum(mass_matrix)
        elif rule=='average':
            final_mass=avg_sum(mass_matrix)
        elif rule=='PCR5':
            final_mass=PCR5_sum(mass_matrix)
        elif rule=='PCR6':
            final_mass=PCR6_sum(mass_matrix)
     
        final_mass=[-1 if v is None else v for v in final_mass]
        final_mass=[-1 if math.isnan(x) else x for x in final_mass]
        
        #print final_mass
        #print np.argmax(final_mass)
        label.append(np.argmax(final_mass))
#         M=3
#         final_m1=find_mi(target,mass_,0)
#         m1_w=1-final_m1
#         final_m2=find_mi(target,mass_,1)
#         m2_w=1-final_m2
#         final_m3=find_mi(target,mass_,2)
#         m3_w=1-final_m3
#         # Convert it to pignistic probability  BetP=m(wq)+m(omega)/M, M= # of classes 
#         pm1=final_m1+m1_w/M
#         pm2=final_m2+m2_w/M
#         pm3=final_m3+m3_w/M
        
#         if pm1 >pm2 and pm1>pm3:
#             label.append(1)
#         elif pm2 >pm1 and pm2>pm3:
#             label.append(2)
#         elif pm3 >pm1 and pm3>pm2:
#             label.append(3)

# Or simply use the max mass for decision making
        
#         if max([final_m1,final_m2,final_m3])==final_m1:
#             label.append(0)
            
#         elif max([final_m1,final_m2,final_m3])==final_m2:
#             label.append(1)
#         elif max([final_m1,final_m2,final_m3])==final_m3:
#             label.append(2)
            
    return label

In [10]:
#### Test
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
import math
def DS_KNN(training,test,ave_dist,metric,rule,k): # training data should be dataframe and also include target
    label=[]
    X_test2=test.iloc[:,1:]
    processed_test=[]
    for i in range(len(X_test2)):
        processed_test.append(np.array(X_test2.iloc[i]))
    processed_test=np.array(processed_test)
    for pt in processed_test:
        dist=distance.cdist([pt], training.iloc[:,1:], metric)
        dist_order=np.argsort(dist)
        best_k_index=dist_order[0,:k]
        nn_k=training.iloc[best_k_index,1:]
        
        target=training.iloc[best_k_index,0]
        target=np.array(target)
        
        target=target.astype(int)
        gamma=ave_dist[target]# target is 0 and 1 so it is changed accordingly        
        mass_=0.95*np.exp(-1.0/gamma*dist[0,best_k_index]**2)
        #k=3
        q=len(original_data.iloc[:,0].unique())
        mass_matrix=np.zeros((k,q))
        for i in range(len(mass_)):
            mass_matrix[i,[target[i]]]=mass_[i]
        
        if rule=='orthogonal':
            final_mass=orthogonal_sum(mass_matrix)
        elif rule=='yager':
            final_mass=yager_sum(mass_matrix)
        elif rule=='average':
            final_mass=avg_sum(mass_matrix)
        elif rule=='PCR5':
            final_mass=PCR5_sum(mass_matrix)
        elif rule=='PCR6':
            final_mass=PCR6_sum(mass_matrix)
     
        final_mass=[-1 if v is None else v for v in final_mass]
        final_mass=[-1 if math.isnan(x) else x for x in final_mass]
        
        #print final_mass
        #print np.argmax(final_mass)
        label.append(np.argmax(final_mass))
#         M=3
#         final_m1=find_mi(target,mass_,0)
#         m1_w=1-final_m1
#         final_m2=find_mi(target,mass_,1)
#         m2_w=1-final_m2
#         final_m3=find_mi(target,mass_,2)
#         m3_w=1-final_m3
#         # Convert it to pignistic probability  BetP=m(wq)+m(omega)/M, M= # of classes 
#         pm1=final_m1+m1_w/M
#         pm2=final_m2+m2_w/M
#         pm3=final_m3+m3_w/M
        
#         if pm1 >pm2 and pm1>pm3:
#             label.append(1)
#         elif pm2 >pm1 and pm2>pm3:
#             label.append(2)
#         elif pm3 >pm1 and pm3>pm2:
#             label.append(3)

# Or simply use the max mass for decision making
        
#         if max([final_m1,final_m2,final_m3])==final_m1:
#             label.append(0)
            
#         elif max([final_m1,final_m2,final_m3])==final_m2:
#             label.append(1)
#         elif max([final_m1,final_m2,final_m3])==final_m3:
#             label.append(2)
            
    return label

In [14]:
#   Back Up
def DSKNN_CV(data,rule):
    X = data
    y = data.iloc[:,0]
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8,random_state=1)
    sss.split(X, y)
    ave_error=[]
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y[train_index], y[test_index]

        distance_metric='euclidean'   ########################## 
        output_type=original_data.iloc[:,0].unique()
        output_type=sorted(output_type)
        ave_dist=[]
        for i in range(len(output_type)):
            temp_data=X_train.loc[X_train.iloc[:,0] == output_type[i]]
            temp_ave_dist=np.sum(pairwise_distances(temp_data.iloc[:,1:],metric=distance_metric))/(len(temp_data)**2-len(temp_data))
            ave_dist.append(temp_ave_dist)
        ave_dist=np.array(ave_dist)    
#         data1=X_train.loc[X_train.iloc[:,0] == 0]
#         data2=X_train.loc[X_train.iloc[:,0] == 1]
#         #data3=X_train.loc[X_train.iloc[:,0] == 2]



#         distance_metric='euclidean'   ########################## 

#         ave_dist1=np.sum(pairwise_distances(data1.iloc[:,1:],metric=distance_metric))/(len(data1)**2-len(data1))
#         ave_dist2=np.sum(pairwise_distances(data2.iloc[:,1:],metric=distance_metric))/(len(data2)**2-len(data2))
#         #ave_dist3=np.sum(pairwise_distances(data3.iloc[:,1:],metric=distance_metric))/(len(data3)**2-len(data3))
#         ave_dist=np.array([ave_dist1,ave_dist2])


#         M1=find_medoid(data1,distance_metric)
#         M2=find_medoid(data2,distance_metric)
#         M3=find_medoid(data3,distance_metric)

        a=DS_KNN(X_train,X_test,ave_dist,distance_metric,rule)
        #print np.array(a).shape,y_test.shape
        print 'error=',len(np.nonzero(a-y_test)[0]),'error rate=',len(np.nonzero(a-y_test)[0])/1.0/len(a)
        #print a
        #print y_test
        ave_error.append(len(np.nonzero(a-y_test)[0])/1.0/len(a))
    ave_error=np.array(ave_error)
    ave_error=np.mean(ave_error)
    print rule,ave_error
    return

In [11]:
#   test ###################################################
def DSKNN_CV(data,rule,k):
    X = data
    y = data.iloc[:,0]
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8,random_state=1)
    sss.split(X, y)
    ave_error=[]
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y[train_index], y[test_index]

        distance_metric='euclidean'   ########################## 
        output_type=original_data.iloc[:,0].unique()
        output_type=sorted(output_type)
        ave_dist=[]
        for i in range(len(output_type)):
            temp_data=X_train.loc[X_train.iloc[:,0] == output_type[i]]
            temp_ave_dist=np.sum(pairwise_distances(temp_data.iloc[:,1:],metric=distance_metric))/(len(temp_data)**2-len(temp_data))
            ave_dist.append(temp_ave_dist)
        ave_dist=np.array(ave_dist)    


        a=DS_KNN(X_train,X_test,ave_dist,distance_metric,rule,k)
        #print np.array(a).shape,y_test.shape
        ##print 'error=',len(np.nonzero(a-y_test)[0]),'error rate=',len(np.nonzero(a-y_test)[0])/1.0/len(a)
        #print a
        #print y_test
        ave_error.append(len(np.nonzero(a-y_test)[0])/1.0/len(a))
    ave_error=np.array(ave_error)
    ave_error=np.mean(ave_error)
    #print rule,ave_error
    return ave_error

In [12]:
def EKNN_classifier(trainset,testset,rule,k):
        distance_metric='euclidean'   ########################## 
        output_type=trainset.iloc[:,0].unique()
        output_type=sorted(output_type)
        ave_dist=[]
        for i in range(len(output_type)):
            temp_data=trainset.loc[trainset.iloc[:,0] == output_type[i]]
            temp_ave_dist=np.sum(pairwise_distances(temp_data.iloc[:,1:],metric=distance_metric))/(len(temp_data)**2-len(temp_data))
            ave_dist.append(temp_ave_dist)
        ave_dist=np.array(ave_dist)    


        prediction=DS_KNN(trainset,testset,ave_dist,distance_metric,rule,k)
        error_rate=1-accuracy_score(testset.iloc[:,0],prediction)
        return error_rate
        

In [13]:
def find_k(data,rule):
    k_list=np.array([1,3,5,7,9,11,13,15,17,19,21,23,25,27,29])
    #k_list=np.array([1,3])
    error_list=[]
    for k in k_list:
        cv_error=DSKNN_CV(data,rule,k)
        error_list.append(cv_error)
    error_list=np.array(error_list)
    best_k=k_list[np.argmin(error_list)]
    print 'best k=',best_k
    return best_k

In [14]:
def EKNN_model(trainset,testset,rule):
    best_k=find_k(trainset,rule)
    err_rate=EKNN_classifier(trainset,testset,rule,best_k)
    print rule,err_rate
    return

In [45]:
original_data=scipy.io.arff.loadarff('EEG Eye State.arff')
original_data=pd.DataFrame(data=original_data[0])
temp1=original_data.iloc[:,:-1]
temp2=original_data.iloc[:,-1]
temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
original_data=pd.concat([temp2,temp1],axis=1)

original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 1
orthogonal 0.0210280373832
best k= 1
yager 0.0210280373832
best k= 1
average 0.0210280373832


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 1
PCR5 0.0210280373832


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 1
PCR6 0.0210280373832


In [46]:
EKNN_model(original_data,original_data,'PCR6')

D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 1
PCR6 0.0


In [117]:
original_data=scipy.io.arff.loadarff('EEG Eye State.arff')
original_data=pd.DataFrame(data=original_data[0])
temp1=original_data.iloc[:,:-1]
temp2=original_data.iloc[:,-1]
temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
original_data=pd.concat([temp2,temp1],axis=1)

DSKNN_CV(original_data,'orthogonal')
DSKNN_CV(original_data,'yager')
DSKNN_CV(original_data,'average')
DSKNN_CV(original_data,'PCR5')
DSKNN_CV(original_data,'PCR6')

error= 166 error rate= 0.0554072096128
error= 180 error rate= 0.0600801068091
error= 179 error rate= 0.0597463284379
error= 147 error rate= 0.0490654205607
error= 172 error rate= 0.0574098798398
orthogonal 0.0563417890521
error= 166 error rate= 0.0554072096128
error= 180 error rate= 0.0600801068091
error= 179 error rate= 0.0597463284379
error= 147 error rate= 0.0490654205607
error= 172 error rate= 0.0574098798398
yager 0.0563417890521
error= 84 error rate= 0.0280373831776
error= 101 error rate= 0.0337116154873
error= 80 error rate= 0.0267022696929
error= 86 error rate= 0.0287049399199
error= 80 error rate= 0.0267022696929
average 0.0287716955941


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


error= 103 error rate= 0.0343791722296
error= 88 error rate= 0.0293724966622
error= 89 error rate= 0.0297062750334
error= 76 error rate= 0.0253671562083
error= 89 error rate= 0.0297062750334
PCR5 0.0297062750334


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


error= 166 error rate= 0.0554072096128
error= 180 error rate= 0.0600801068091
error= 179 error rate= 0.0597463284379
error= 147 error rate= 0.0490654205607
error= 172 error rate= 0.0574098798398
PCR6 0.0563417890521


In [47]:
original_data=pd.read_csv('ionosphere.data')
# bad=0, good=1
original_data['g'] = original_data['g'].astype('category')
original_data['g'] = original_data['g'].cat.codes
temp1=original_data.iloc[:,:-1]
temp2=original_data.iloc[:,-1]
temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
original_data=pd.concat([temp2,temp1],axis=1)
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 25
orthogonal 0.342857142857
best k= 25
yager 0.342857142857
best k= 1
average 0.185714285714


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 1
PCR5 0.185714285714


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 25
PCR6 0.342857142857


In [48]:
original_data=pd.read_csv('wine.data', sep=',',header=None)
original_data.iloc[:,0]=original_data.iloc[:,0]-1
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 19
orthogonal 0.666666666667
best k= 19
yager 0.666666666667
best k= 9
average 0.25


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 21
PCR5 0.666666666667


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 21
PCR6 0.666666666667


In [49]:
temp1=pd.read_csv('madelon_train.data',header=None)
aa=[]
for i in range(len(temp1.iloc[:,0])):
    aa.append(map(float, temp1.iloc[:,0][i].split()))
temp1=pd.DataFrame(data=aa)

temp1=(temp1-temp1.min())/(temp1.max()-temp1.min())
temp2=pd.read_csv('madelon_train.labels',header=None)
temp2.iloc[np.where(temp2.iloc[:,0]<0)[0],0]=0
# # # temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
dataset=pd.concat([temp2,temp1],axis=1)
dataset=dataset.astype('float')

original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 1
orthogonal 0.4425
best k= 1
yager 0.4425
best k= 3
average 0.46


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 3
PCR5 0.46


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 1
PCR6 0.4425


In [50]:
dataset=pd.read_csv('iris.data',header=None)
dataset.iloc[:,-1]= dataset.iloc[:,-1].astype('category')
dataset.iloc[:,-1]=dataset.iloc[:,-1].cat.codes

temp1=dataset.iloc[:,:-1]
temp2=dataset.iloc[:,-1]
# # temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
dataset=pd.concat([temp2,temp1],axis=1)
original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 11
orthogonal 0.266666666667
best k= 11
yager 0.266666666667
best k= 1
average 0.0333333333333


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 7
PCR5 0.0666666666667


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 11
PCR6 0.266666666667


In [51]:
dataset=pd.read_csv('dataR2.csv')
dataset.iloc[:,-1]= dataset.iloc[:,-1].astype('category')
dataset.iloc[:,-1]=dataset.iloc[:,-1].cat.codes

temp1=dataset.iloc[:,:-1]
temp2=dataset.iloc[:,-1]
# # # temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
dataset=pd.concat([temp2,temp1],axis=1)
original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 1
orthogonal 0.333333333333
best k= 1
yager 0.333333333333
best k= 1
average 0.333333333333


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 1
PCR5 0.333333333333


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 1
PCR6 0.333333333333


In [52]:
dataset=pd.read_csv('haberman.data',header=None) 
temp1=dataset.iloc[:,:-1]
temp2=dataset.iloc[:,-1]-1
dataset=pd.concat([temp2,temp1],axis=1)
original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 5
orthogonal 0.354838709677
best k= 5
yager 0.354838709677
best k= 19
average 0.370967741935


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 9
PCR5 0.354838709677


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 5
PCR6 0.354838709677


In [53]:
dataset=pd.read_csv('SPECT.test',header=None)
original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 1
orthogonal 0.0263157894737
best k= 1
yager 0.0263157894737
best k= 13
average 0.0789473684211


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 3
PCR5 0.0789473684211


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 1
PCR6 0.0263157894737


In [54]:
dataset=pd.read_csv('data_banknote_authentication.txt')
dataset
temp1=dataset.iloc[:,:-1]
temp2=dataset.iloc[:,-1]
dataset=pd.concat([temp2,temp1],axis=1)
original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 29
orthogonal 0.0218181818182
best k= 29
yager 0.0218181818182
best k= 1
average 0.00363636363636


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 29
PCR5 0.0181818181818


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 29
PCR6 0.0218181818182


In [55]:
dataset=pd.read_csv('tae.data',header=None)
dataset.iloc[:,-1]= dataset.iloc[:,-1].astype('category')
dataset.iloc[:,-1]=dataset.iloc[:,-1].cat.codes

temp1=dataset.iloc[:,:-1]
temp2=dataset.iloc[:,-1]
dataset=pd.concat([temp2,temp1],axis=1)
original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 1
orthogonal 0.483870967742
best k= 1
yager 0.483870967742
best k= 5
average 0.548387096774


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 1
PCR5 0.483870967742


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 1
PCR6 0.483870967742


In [56]:
dataset=pd.read_csv('wdbc.data',header=None)
dataset=dataset.iloc[:,1:]
dataset.iloc[:,0]= dataset.iloc[:,0].astype('category')
dataset.iloc[:,0]=dataset.iloc[:,0].cat.codes
original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

best k= 25
orthogonal 0.245614035088
best k= 25
yager 0.245614035088
best k= 1
average 0.0877192982456


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 25
PCR5 0.140350877193


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 25
PCR6 0.245614035088


In [17]:
#Diabetic Retinopathy Debrecen 
original_data=scipy.io.arff.loadarff('messidor_features.arff')
original_data=pd.DataFrame(data=original_data[0])
temp1=original_data.iloc[:,:-1]
temp2=original_data.iloc[:,-1]
temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
original_data=pd.concat([temp2,temp1],axis=1)

original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')


best k= 1
orthogonal 0.393939393939
best k= 1
yager 0.393939393939
best k= 23
average 0.398268398268


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in divide
  """


best k= 1
PCR5 0.393939393939


D:\Anaconda\python2.7\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


best k= 1
PCR6 0.393939393939


In [34]:
dataset=pd.read_csv('occupancy_train.txt')
dataset=dataset.iloc[:,1:]
#dataset.iloc[:,-1]= dataset.iloc[:,-1].astype('category').cat.codes
#dataset.iloc[:,-1]=dataset.iloc[:,0].cat

temp1=dataset.iloc[:,:-1]
temp2=dataset.iloc[:,-1]
# #temp2 = pd.to_numeric(temp2, errors='coerce').fillna(0).astype(np.int64)
original_data=pd.concat([temp2,temp1],axis=1)

original_data

original_data=dataset
original_data,testset= train_test_split(original_data, test_size=0.2)
EKNN_model(original_data,testset,'orthogonal')
EKNN_model(original_data,testset,'yager')
EKNN_model(original_data,testset,'average')
EKNN_model(original_data,testset,'PCR5')
EKNN_model(original_data,testset,'PCR6')

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [93]:
# This cell is for back up
from sklearn.neighbors import NearestNeighbors
from scipy.spatial import distance
import math
def DS_KNN(training,test,ave_dist,metric): # training data should be dataframe and also include target
    label=[]
    X_test2=test.iloc[:,1:]
    processed_test=[]
    for i in range(len(X_test2)):
        processed_test.append(np.array(X_test2.iloc[i]))
    processed_test=np.array(processed_test)
    for pt in processed_test:
        dist=distance.cdist([pt], training.iloc[:,1:], metric)
        dist_order=np.argsort(dist)
        best_3_index=dist_order[0,:3]
        nn_3=training.iloc[best_3_index,1:]
        
        target=training.iloc[best_3_index,0]
        target=np.array(target)
        
        target=target.astype(int)
        gamma=ave_dist[target]# target is 0 and 1 so it is changed accordingly        
        mass_=0.95*np.exp(-1.0/gamma*dist[0,best_3_index]**2)
        k=3
        q=2
        mass_matrix=np.zeros((k,q))
        for i in range(len(mass_)):
            mass_matrix[i,[target[i]]]=mass_[i]
        
        #final_mass=orthogonal_sum(mass_matrix)
        #final_mass=yager_sum(mass_matrix)
        #final_mass=avg_sum(mass_matrix)
        #final_mass=PCR5_sum(mass_matrix)
        final_mass=PCR6_sum(mass_matrix)
        final_mass=[-1 if v is None else v for v in final_mass]
        final_mass=[-1 if math.isnan(x) else x for x in final_mass]
        
        #print final_mass
        #print np.argmax(final_mass)
        label.append(np.argmax(final_mass))
#         M=3
#         final_m1=find_mi(target,mass_,0)
#         m1_w=1-final_m1
#         final_m2=find_mi(target,mass_,1)
#         m2_w=1-final_m2
#         final_m3=find_mi(target,mass_,2)
#         m3_w=1-final_m3
#         # Convert it to pignistic probability  BetP=m(wq)+m(omega)/M, M= # of classes 
#         pm1=final_m1+m1_w/M
#         pm2=final_m2+m2_w/M
#         pm3=final_m3+m3_w/M
        
#         if pm1 >pm2 and pm1>pm3:
#             label.append(1)
#         elif pm2 >pm1 and pm2>pm3:
#             label.append(2)
#         elif pm3 >pm1 and pm3>pm2:
#             label.append(3)

# Or simply use the max mass for decision making
        
#         if max([final_m1,final_m2,final_m3])==final_m1:
#             label.append(0)
            
#         elif max([final_m1,final_m2,final_m3])==final_m2:
#             label.append(1)
#         elif max([final_m1,final_m2,final_m3])==final_m3:
#             label.append(2)
            
    return label

In [78]:
# This cell is for back-up 
def DSKNN_CV(data):
    X = data
    y = data.iloc[:,0]
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, train_size=0.8,random_state=1)
    sss.split(X, y)
    ave_error=[]
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y[train_index], y[test_index]


        data1=X_train.loc[X_train.iloc[:,0] == 0]
        data2=X_train.loc[X_train.iloc[:,0] == 1]
        #data3=X_train.loc[X_train.iloc[:,0] == 2]



        distance_metric='euclidean'   ########################## 

        ave_dist1=np.sum(pairwise_distances(data1.iloc[:,1:],metric=distance_metric))/(len(data1)**2-len(data1))
        ave_dist2=np.sum(pairwise_distances(data2.iloc[:,1:],metric=distance_metric))/(len(data2)**2-len(data2))
        #ave_dist3=np.sum(pairwise_distances(data3.iloc[:,1:],metric=distance_metric))/(len(data3)**2-len(data3))
        ave_dist=np.array([ave_dist1,ave_dist2])


#         M1=find_medoid(data1,distance_metric)
#         M2=find_medoid(data2,distance_metric)
#         M3=find_medoid(data3,distance_metric)

        a=DS_KNN(X_train,X_test,ave_dist,distance_metric)
        #print np.array(a).shape,y_test.shape
        print 'error=',len(np.nonzero(a-y_test)[0]),'error rate=',len(np.nonzero(a-y_test)[0])/1.0/len(a)
        #print a
        #print y_test
        ave_error.append(len(np.nonzero(a-y_test)[0])/1.0/len(a))
    ave_error=np.array(ave_error)
    ave_error=np.mean(ave_error)
    print ave_error
    return